In [1]:
# Kathleen Beilsmith
# January 2020
# Argonne National Lab

# Notebook: Import metabolic models from KBase and convert to cobrapy

In [1]:
# Import required modules
import platform
print("python version " + platform.python_version())
import os
import logging
import optlang
import re
from optlang.symbolics import Zero, add
from IPython.core.display import HTML
#from configparser import ConfigParser
#config = ConfigParser()
#f = open("config.cfg", "r")
#print(f.read())
#config.read("config.cfg")
#paths = config.get("script", "syspaths").split(";")
#for path in paths:
#    sys.path.append(path)

# cobra
import cobra
from cobra.core.dictlist import DictList
from cobra.core import Gene, Metabolite, Model, Reaction
import cobra.util.solver as sutil

# cobrakbase
import cobrakbase
from cobrakbase.core.converters import KBaseFBAModelToCobraBuilder
from cobrakbase.Workspace.WorkspaceClient import Workspace as WorkspaceClient
from cobrakbase.core.kbase_object_factory import KBaseObjectFactory
from cobrakbase.core.fba_utilities import KBaseFBAUtilities
from modelseedpy import MSPackageManager, MSExpression
from MSExpression import LOWEST

# solver
import cplex

# dataframes
import pandas

# data import/export
import json
import csv

# Print when finished.
print("Required modules loaded")

python version 3.7.9
cobrakbase 0.2.7
Required modules loaded


In [2]:
#Loading expression data for our conditions
genome = kbase_api.get_from_ws("Bacteroides_thetaiotaomicron_VPI-5482",81169)
gene_expression = MSExpression.from_gene_feature_file("ExpressionData.tsv",genome,False)

#Loading published metabolic model
kbase_api = cobrakbase.KBaseAPI()
model = kbase_api.get_from_ws('iAH991', 81169)

#Translating gene expression to reaction expression
rxn_expression = gene_expression.build_reaction_expression(model)

#Loading media that mimicks innoculation media
innoculation_media = kbase_api.get_from_ws("ArgonneLBMedia","KBaseMedia")
day_one_media = kbase_api.get_from_ws("MouseDayOne",84574)#Send chris list of media composition
day_seven_media = kbase_api.get_from_ws("MouseDaySeven",84574)
day_forteen_media = kbase_api.get_from_ws("MouseDayForteen",84574)

Shewanella_oneidensis_MR-1_Gapfilled_Lactate


In [ ]:
#Create flux profile that is as close to innoculation growth as possible
#First get ModelSEEDpy package manager so we can add constraints to our model
pkgmgr = MSPackageManager.get_pkg_mgr(model,1)
#Add the innoculation media as the growth condition
pkgmgr.getpkg("KBaseMediaPkg").build_package(innoculation_media)
#Set a limit on total carbon uptake by the model
pkgmgr.getpkg("ElementUptakePkg").build_package({"C":60})
#Set the known growth rate for B. theta in this condition
biomass = model.reactions.get_by_id("bio1")
double_time = ?
growth_rate = #Chris will fill this in
biomass.upper_bound = growth_rate
biomass.lower_bound = growth_rate
#Now we minimize all fluxes based on expression data (penalize higher expressed reactions less)
#Minimize sum of reaction fluxes with coefficients based on expression data
#Save reference flux

sol=model.optimize()
model.summary()

In [3]:
#Next we simulate the day 1 condition
#Set the media constraint
pkgmgr.getpkg("KBaseMediaPkg").build_package(day_one_media)
pkgmgr.getpkg("FlexibleBiomassPkg",1).build_package({"bio_rxn_id":"bio1","use_rna_class":[-0.99999,0.99999],
    "use_dna_class":[-0.25,0.25],
    "use_protein_class":[-0.25,0.25],
    "use_energy_class":[-0.1,0.1]})
#Release constraints on the growth rate (we want to see if we can predict this)
biomass.upper_bound = 100
biomass.lower_bound = 0
#Now we predict flux change from fold change in expression data
newflux = {}
for rxn in expression:
    newflux[rxn] = expression[rxn].foldchange*reference_flux[rxn]
#Now we fit the flux to the fold change fluxes

#Print results
sol=model.optimize()
model.summary()
#Simulate all single gene KO



solver: cplex
tolerance: 1e-07
lower_bound: -1000.0
upper_bound: 1000.0
processes: 15
cache_directory: /Users/kbeilsmith/Library/Caches/cobrapy
max_cache_size: 104857600
cache_expiration: None



In [4]:
# Another check, with Megan's Btheta model.

kbase_api = cobrakbase.KBaseAPI()

kbmodel = kbase_api.get_from_ws('iAH991.xml_model', 84574)

# Check that it worked.

print(kbmodel)

iAH991.xml_model


In [5]:
# Hurrah! What's next?